# H1: Human 7DoF Reach and Grasp

H1 is a dataset that was collected for open loop calibration of a BCI for robot arm control. The dataset contains neural activity of a human participant as they attempted to reach and grasp their right hand according to visually cued motion for 7 degrees of freedom (DoF). The visual cue was a screen illustrating the desired motion on a virtual arm, and the participant has had practice following these cues to calibrate similar decoders before this dataset was collected.

# TODO gross statistics, trialized structure..

## Dataset statistics
- 7 DoF: 3D position of the hand, 3D orientation of the hand, 1D grasp aperture
    - 50Hz upsampled to 100Hz
- 176 motor units. Thresholds determined on daily basis.
- Train, short, and long test phases illustrated on dataset timelines

## Trial structure

## Neural activity
Multi-unit activity